In [56]:
import torch
from torch import nn, optim 
from torch.utils.data import DataLoader 
from torchvision.models import vgg16
from torchvision import datasets, transforms 
from tqdm import tqdm

In [57]:
BATCH_SIZE = 32
IMG_SIZE = 224 # input image size for VGG-16

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])
data = datasets.CIFAR10(root = '~/Desktop/data', train=True, download=True, transform=transform)
loader = DataLoader(dataset=data, batch_size=BATCH_SIZE, shuffle=True) # train loader

test = datasets.CIFAR10(root = '~/Desktop/data', train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test, batch_size=BATCH_SIZE, shuffle=True)

dev = torch.device('mps')

Files already downloaded and verified
Files already downloaded and verified


In [58]:
model = vgg16(pretrained=True).to(dev) # loading pre-trained model
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [59]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=10).to(dev) # replacing output layer

parameters_to_update = []
parameter_names = ['classifier.6.weight', 'classifier.6.bias'] # names of parameters to be updated

for name, parameter in model.named_parameters():
    if name in parameter_names :
        parameter.requires_grad = True # accumulates gradients for required parameterss
        parameters_to_update.append(parameter) # adds param to update list for optimizer
    else : 
        parameter.requires_grad = False # rest of the parameters are not updated


LR = 0.001
EPOCHS = 5

crit = nn.CrossEntropyLoss()
opt = optim.Adam(params=parameters_to_update, lr=LR)

In [60]:
ALL_LOSS = []

for e in range(EPOCHS):

    loop = tqdm(enumerate(loader), total=len(loader), leave=False, position=0)
    model.train()

    for batch_id, (x, y) in loop : 

        x, y = x.to(dev), y.to(dev)
        opt.zero_grad() 
        yhat = model(x)
        loss = crit(yhat, y)
        ALL_LOSS.append(loss.item())
        loss.backward()
        opt.step()

        loop.set_description(f'Epoch : [{e}/{EPOCHS}]')
        loop.set_postfix(loss=loss.item())


    loop = tqdm(enumerate(test_loader), total=len(test_loader), leave=False, position=0)
    model.eval()
    correct = 0

    for batch_id, (x, y) in loop :    
        x, y = x.to(dev), y.to(dev) 
        yhat = model(x).argmax(dim=1)
        correct += torch.sum(yhat == y).item()
    acc = correct / len(test)
    print(f"Accuracy : {acc}")

Accuracy : 0.6932


Accuracy : 0.7086


Accuracy : 0.7096


Accuracy : 0.7089


Accuracy : 0.7224
